# 쇼핑몰 리뷰 감성 분석: Full Fine-Tuning vs PEFT (LoRA)

## 1. 프로젝트 개요
- **목표**: 패션 도메인 리뷰의 감성(긍정/중립/부정)을 분석하는 모델 최적화.
- **Base Model**: `klue/roberta-base`
- **비교 대상**: Full Fine-Tuning (전체 학습) vs LoRA (효율적 학습)
- **환경**: Local (M1 Debugging) / Colab (Full Training)

In [ ]:
# =========================================================
# 프로젝트 폴더 동적 탐색 및 셋업
# =========================================================
import sys
import os
from pathlib import Path

# 1. Colab 환경 감지 및 드라이브 마운트
if 'google.colab' in sys.modules:
    print("Google Colab 환경 감지됨. 드라이브 연결 중...")

    # 라이브러리 설치
    !pip install -q transformers peft datasets evaluate accelerate

    # 드라이브 마운트
    from google.colab import drive
    drive.mount('/content/drive')

    # [핵심] 프로젝트 폴더 이름으로 위치 찾기
    # 하드코딩 없이 '13_SentimentAnalysis_Project'라는 이름만 찾습니다.
    TARGET_DIR_NAME = "13_SentimentAnalysis_Project"
    SEARCH_ROOT = Path("/content/drive/MyDrive")

    print(f"구글 드라이브에서 '{TARGET_DIR_NAME}' 폴더를 검색합니다...")

    found_path = None

    # 1단계: 빠른 검색 (MyDrive 바로 아래 혹은 1단계 깊이 폴더 확인)
    # 예: MyDrive/13_... 또는 MyDrive/codeit/13_...
    try:
        # rglob은 너무 느릴 수 있으므로, 2 depth까지만 명시적으로 확인
        patterns = [
            f"{TARGET_DIR_NAME}",           # Root
            f"*/{TARGET_DIR_NAME}",         # Depth 1 (codeit/...)
            f"*/*/{TARGET_DIR_NAME}"        # Depth 2
        ]

        for pattern in patterns:
            results = list(SEARCH_ROOT.glob(pattern))
            if results:
                found_path = results[0]
                break
    except Exception as e:
        print(f"검색 중 오류: {e}")

    # 이동 및 결과 확인
    if found_path and found_path.exists():
        os.chdir(found_path)
        print(f"작업 디렉토리 변경 완료: {os.getcwd()}")
    else:
        raise FileNotFoundError(f"구글 드라이브에서 '{TARGET_DIR_NAME}' 폴더를 찾을 수 없습니다.")

else:
    print("로컬(Mac M1) 환경 감지됨. 현재 위치 유지.")

# 2. 시스템 경로 설정 (src 모듈용)
current_path = Path(os.getcwd())
# notebooks 폴더 안에 있다면 상위로 이동
project_root = current_path.parent if current_path.name == 'notebooks' else current_path

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Python 경로 추가됨: {project_root}")

# 3. 데이터 로드 사전 점검 (제공해주신 경로 기준)
# 경로: data/쇼핑몰/01. 패션/1-1. 여성의류/...
target_check = Path("./data/쇼핑몰")
if target_check.exists():
    print("데이터 폴더 확인됨.")
    # 실제 파일이 있는지 간단 체크
    fashion_check = list(target_check.glob("**/*패션*/**/*.json"))
    print(f"패션 카테고리 JSON 파일 발견 수: {len(fashion_check)}개")
else:
    print("경고: './data/쇼핑몰' 경로를 찾을 수 없습니다.")

Google Colab 환경 감지됨. 드라이브 연결 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
구글 드라이브에서 '13_SentimentAnalysis_Project' 폴더를 검색합니다...
작업 디렉토리 변경 완료: /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project
데이터 폴더 확인됨.
패션 카테고리 JSON 파일 발견 수: 0개


In [ ]:
PROJECT_PATH

'/content/drive/MyDrive/codeit/13_SentimentAnalysis_Project'

/content/drive/MyDrive/codeit/13_SentimentAnalysis_Project

## 2. 유틸리티 및 데이터 로더 (Self-Contained)
데이터 누수 방지를 위한 Group Split 로직이 포함되어 있습니다.

In [ ]:
# ---------------------------------------------------------
# 1. 라이브러리 임포트 및 로깅 설정
# ---------------------------------------------------------
import sys
import os
import json
import logging
import pandas as pd
import torch
import numpy as np
import unicodedata
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import PeftModel, LoraConfig, get_peft_model, TaskType
import evaluate

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------------------------------------
# 2. 장치(Device) 감지
# ---------------------------------------------------------
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
logger.info(f"Using device: {device}")

is_mac_local = (device.type == 'mps')
is_colab = ('google.colab' in sys.modules)

if is_mac_local:
    logger.warning("Running on Mac M1 (MPS). Training will be limited to debugging samples.")

In [ ]:
# ---------------------------------------------------------
# 3. 데이터 로드 및 경로 탐색
# ---------------------------------------------------------
import unicodedata
import logging
import sys
from pathlib import Path
from src.data_loader import load_json_files, parse_data, split_data_leakage_proof, create_dataset_dict

# 경로 찾기 함수 (현재 위치 ./data 기준)
def find_target_data_dir():
    # 앞선 셋업 셀에서 os.chdir로 프로젝트 루트로 이동했다고 가정합니다.
    base_data_path = Path("./data")

    if not base_data_path.exists():
        raise FileNotFoundError(f"현재 위치({Path.cwd()})에 'data' 폴더가 없습니다. [통합 셋업] 셀을 먼저 실행했는지 확인해주세요.")

    # 1. '쇼핑몰' 폴더 찾기 (이름이 조금 달라도 찾을 수 있게 glob 사용)
    try:
        shopping_root = next(base_data_path.glob("*쇼핑몰*"))
    except StopIteration:
        raise FileNotFoundError("'data' 폴더 내에서 '쇼핑몰' 폴더를 찾을 수 없습니다.")

    # 2. 타겟 카테고리 찾기 (패션 우선 탐색)
    # 자소분리(NFD) 대응을 위해 normalize 사용
    target_keywords = ["패션", "화장품", "가전", "IT기기"]

    for keyword in target_keywords:
        for path in shopping_root.glob("*"):
            # Mac(NFD)과 Linux(NFC)의 한글 인코딩 차이 해결
            if keyword in unicodedata.normalize('NFC', path.name):
                logger.info(f"분석 대상 카테고리 발견: {path.name}")
                return path

    raise FileNotFoundError(f"지정된 카테고리({target_keywords}) 폴더를 찾을 수 없습니다.")

# --- 실행 ---
try:
    # 1. 경로 확정
    DATA_DIR = find_target_data_dir()
    print(f"최종 데이터 경로: {DATA_DIR}")

    # 2. 로드 샘플링 설정
    # 이전 셀에서 is_mac_local이 정의되지 않았을 경우를 대비해 안전하게 처리
    is_mac_local = globals().get('is_mac_local', False)
    limit = 2000 if is_mac_local else None

    if is_mac_local:
        print(f"로컬 환경 감지: 데이터 로드를 {limit}건으로 제한합니다.")

    # 3. src 모듈 함수 호출
    files = load_json_files(DATA_DIR)
    if not files:
        raise ValueError("JSON 파일이 없습니다.")

    # 파싱 (src/data_loader.py의 수정된 로직이 적용됨)
    df = parse_data(files, sample_limit=limit)

    # 누수 방지 분할
    train_df, test_df = split_data_leakage_proof(df)

    # DatasetDict 생성
    dataset = create_dataset_dict(train_df, test_df)

    print("\n데이터셋 준비 완료:")
    print(dataset)

except Exception as e:
    print(f"에러 발생: {e}")
    # 디버깅을 위해 상세 정보 출력
    import traceback
    traceback.print_exc()

에러 발생: 'data' 폴더 내에서 '쇼핑몰' 폴더를 찾을 수 없습니다.


Traceback (most recent call last):
  File "/tmp/ipython-input-731740249.py", line 20, in find_target_data_dir
    shopping_root = next(base_data_path.glob("*쇼핑몰*"))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipython-input-731740249.py", line 40, in <cell line: 0>
    DATA_DIR = find_target_data_dir()
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-731740249.py", line 22, in find_target_data_dir
    raise FileNotFoundError("'data' 폴더 내에서 '쇼핑몰' 폴더를 찾을 수 없습니다.")
FileNotFoundError: 'data' 폴더 내에서 '쇼핑몰' 폴더를 찾을 수 없습니다.


In [ ]:
import os
from pathlib import Path
import unicodedata

print("CWD:", os.getcwd())
print("CWD list:", [p.name for p in Path(".").iterdir()][:50])

p_data = Path("./data")
p_shop = Path("./data/쇼핑몰")

print("\n./data exists?      ", p_data.exists(), " -> ", p_data.resolve())
print("./data/쇼핑몰 exists?", p_shop.exists(), " -> ", p_shop.resolve())

# 프로젝트 폴더 안에서 data 폴더가 어디 있는지(깊이 4까지) 찾아보기
candidates = list(Path(".").glob("**/data"))
candidates = [c for c in candidates if len(c.parts) <= len(Path(".").parts) + 4]

print("\nFound data candidates (depth<=4):")
for c in candidates[:30]:
    print(" -", c, "(exists:", c.exists(), ")")

# drive 전체(MyDrive)에서 '쇼핑몰' 또는 'data'가 어디 있는지(너무 느리면 아래 2개 중 하나만)
root = Path("/content/drive/MyDrive")
print("\nMyDrive quick scan (depth<=3) for 'data' / '쇼핑몰':")
for pat in ["data", "*data*", "*쇼핑몰*"]:
    hits = list(root.glob(pat)) + list(root.glob(f"*/{pat}")) + list(root.glob(f"*/*/{pat}"))
    hits = [h for h in hits if h.exists()]
    if hits:
        print(f"Pattern {pat}:")
        for h in hits[:10]:
            print("  -", h)

CWD: /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project
CWD list: ['FIRST_ORDER.md', 'MISSION_GUIDE.md', 'README.md', 'requirements.txt', 'INSTRUCTIONS.md', 'data', 'notebooks', 'results_debug', 'logs', 'models', 'src']

./data exists?       True  ->  /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data
./data/쇼핑몰 exists? True  ->  /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data/쇼핑몰

Found data candidates (depth<=4):
 - data (exists: True )

MyDrive quick scan (depth<=3) for 'data' / '쇼핑몰':
Pattern data:
  - /content/drive/MyDrive/codeit/part-1-main/data
  - /content/drive/MyDrive/codeit/NLP/data
  - /content/drive/MyDrive/codeit/12_NLP_Summarization/data
  - /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data
Pattern *data*:
  - /content/drive/MyDrive/codeit/11 data
  - /content/drive/MyDrive/DA_Sample_Projects/샘플17_모빌리티_New York 승객 픽업 분석/uber-raw-data-apr14.csv
  - /content/drive/MyDrive/DA_Sample_Projects/샘플17_ᄆ

In [ ]:
from pathlib import Path

target_check = Path("./data/쇼핑몰")
print("target_check exists:", target_check.exists(), target_check.resolve())

# 1) json 전체 개수(키워드 필터 없이)
all_json = list(target_check.glob("**/*.json"))
print("ALL json count:", len(all_json))
print("sample:", all_json[:5])

# 2) '패션' 들어간 디렉토리 후보 보기
dirs = [p for p in target_check.glob("**/*") if p.is_dir()]
fashion_dirs = [d for d in dirs if "패션" in d.name]
print("fashion dir candidates:", fashion_dirs[:10])

target_check exists: True /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data/쇼핑몰
ALL json count: 1766
sample: [PosixPath('data/쇼핑몰/04. IT기기/4-4. 자동차기기/4-4.자동차기기(65).json'), PosixPath('data/쇼핑몰/04. IT기기/4-4. 자동차기기/4-4.자동차기기(17).json'), PosixPath('data/쇼핑몰/04. IT기기/4-4. 자동차기기/4-4.자동차기기(19).json'), PosixPath('data/쇼핑몰/04. IT기기/4-4. 자동차기기/4-4.자동차기기(100).json'), PosixPath('data/쇼핑몰/04. IT기기/4-4. 자동차기기/4-4.자동차기기(6).json')]
fashion dir candidates: []


In [ ]:
import os

print(f"📍 현재 작업 위치: {os.getcwd()}")
print("\n📂 [현재 폴더의 파일/폴더 목록]")
files = os.listdir(".")
for f in files:
    print(f" - {f}")

print("-" * 30)

# 혹시 data 폴더가 없다면 경고 메시지 출력
if "data" not in files:
    print("🚨 'data' 폴더가 보이지 않습니다!")
    print("   -> 구글 드라이브에 'data' 폴더를 업로드했는지 확인해주세요.")
    print("   -> 혹은 'data' 폴더가 다른 폴더(예: notebooks) 안에 들어있는지 확인해주세요.")
else:
    print("✅ 'data' 폴더가 존재합니다. 내부를 확인합니다.")
    print(os.listdir("./data"))

📍 현재 작업 위치: /content/drive/MyDrive/codeit/13_SentimentAnalysis_Project

📂 [현재 폴더의 파일/폴더 목록]
 - FIRST_ORDER.md
 - MISSION_GUIDE.md
 - README.md
 - requirements.txt
 - INSTRUCTIONS.md
 - data
 - notebooks
 - results_debug
 - logs
 - models
 - src
------------------------------
✅ 'data' 폴더가 존재합니다. 내부를 확인합니다.
['쇼핑몰', 'SNS']


In [ ]:
import sys
import os
import unicodedata
from pathlib import Path
from src.data_loader import load_json_files, parse_data, split_data_leakage_proof, create_dataset_dict
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- [핵심 수정] 한글 자소 분리(NFD) 대응 경로 찾기 함수 ---
def robust_find_data_path(base_path_str="./data", target_keyword="쇼핑몰"):
    base_path = Path(base_path_str)

    if not base_path.exists():
        # 혹시 상대 경로가 안 먹히면 절대 경로로 시도
        base_path = Path(os.getcwd()) / "data"
        if not base_path.exists():
            raise FileNotFoundError(f"🚨 'data' 폴더를 찾을 수 없습니다. 현재 위치: {os.getcwd()}")

    logger.info(f"📂 '{base_path}' 내부를 검색합니다...")

    # data 폴더 안의 모든 폴더를 하나씩 꺼내서 정규화(NFC) 후 확인
    found_path = None
    for child in base_path.iterdir():
        # 폴더 이름 정규화 (Mac NFD -> Linux NFC)
        normalized_name = unicodedata.normalize('NFC', child.name)

        if target_keyword in normalized_name:
            logger.info(f"✅ 타겟 폴더 발견 (NFC 변환됨): {child.name} -> {normalized_name}")
            found_path = child
            break

    if not found_path:
        # 못 찾았으면 목록 출력하여 디버깅
        all_folders = [unicodedata.normalize('NFC', p.name) for p in base_path.iterdir()]
        raise FileNotFoundError(f"'{target_keyword}' 폴더를 찾지 못했습니다. 발견된 폴더: {all_folders}")

    return found_path

# --- 실행 ---
try:
    # 1. 튼튼한 경로 찾기 실행
    shopping_dir = robust_find_data_path(target_keyword="쇼핑몰")
    print(f"🎯 최종 확정된 쇼핑몰 경로: {shopping_dir}")

    # 2. 파일 로드 (재귀적으로 모든 하위 폴더 검색)
    # glob 패턴을 조금 더 유연하게 설정 (경로 중간에 자소 분리된 한글이 있어도 찾도록)
    print("파일 검색을 시작합니다... (시간이 조금 걸릴 수 있습니다)")

    # src/data_loader.py의 load_json_files가 내부적으로 glob을 쓰는데,
    # 여기서 직접 파일 리스트를 구해서 넘겨주는 것이 가장 안전합니다.
    all_json_files = []

    # os.walk를 써서 모든 하위 디렉토리를 샅샅이 뒤짐
    for root, dirs, files in os.walk(shopping_dir):
        for file in files:
            if file.endswith('.json') and '패션' in unicodedata.normalize('NFC', root):
                all_json_files.append(Path(root) / file)

    print(f"📦 발견된 패션 관련 JSON 파일 수: {len(all_json_files)}개")

    if len(all_json_files) == 0:
        raise ValueError("JSON 파일을 찾지 못했습니다. 폴더 구조(패션/...)를 확인해주세요.")

    # 3. 데이터 파싱 및 데이터셋 생성
    # 로컬/테스트 환경이면 2000개만, 아니면 전체
    limit = 2000
    print(f"🚀 데이터 로딩 및 파싱 시작 (샘플링: {limit}개)...")

    # load_json_files 대신 직접 찾은 파일 리스트를 이용하거나,
    # src 모듈이 리스트를 받을 수 없다면 여기서 직접 df 생성
    # (일반적으로 src 코드를 수정할 수 없으니, 여기서 직접 로드 로직 일부 구현)

    # 기존 함수 재활용을 위해 files 리스트를 넘길 수 있는지 확인.
    # 만약 load_json_files가 경로만 받는다면, 우리가 찾은 경로 중 '패션'이 포함된 상위 폴더를 넘겨야 함.
    # 하지만 가장 확실한 건 파일 리스트를 직접 파싱 함수에 넘기는 것입니다.

    # *참고: src.data_loader 구조를 정확히 모르므로,
    # 일반적인 경우인 'parse_data'가 파일 경로 리스트나 내용을 처리한다고 가정하고 진행합니다.*
    # 만약 parse_data가 파일 경로 리스트를 받는다면:
    df = parse_data(all_json_files, sample_limit=limit)

    # 4. 분할 및 데이터셋 생성
    train_df, test_df = split_data_leakage_proof(df)
    dataset = create_dataset_dict(train_df, test_df)

    print("\n✨ 데이터셋 준비 완료!")
    print(dataset)
    print("\nTrain 예시:")
    print(dataset['train'][0])

except Exception as e:
    print(f"\n❌ 에러 발생: {e}")
    import traceback
    traceback.print_exc()

🎯 최종 확정된 쇼핑몰 경로: data/쇼핑몰
파일 검색을 시작합니다... (시간이 조금 걸릴 수 있습니다)
📦 발견된 패션 관련 JSON 파일 수: 450개
🚀 데이터 로딩 및 파싱 시작 (샘플링: 2000개)...

✨ 데이터셋 준비 완료!
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1638
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 454
    })
})

Train 예시:
{'text': '배송도빠르고  상품도 가격대비 가성비 굿~~ 너무 착용하기 편하고 가죽도 부드럽고 너무 좋습니다. 굿굿굿~~너무 맘에 듭니다^^', 'label': 2}


In [ ]:
import json
from pathlib import Path

# 1. 파일이 실제로 잡히는지 확인
print(f"현재 설정된 DATA_DIR: {DATA_DIR}")
all_files = list(Path(DATA_DIR).glob("**/*.json"))
print(f"발견된 JSON 파일 개수: {len(all_files)}")

if len(all_files) == 0:
    print("❌ 파일을 하나도 못 찾았습니다! 경로 문제(자소 분리 등)일 가능성이 큽니다.")
    print("상위 폴더 내용을 출력합니다:")
    !ls -R "{DATA_DIR.parent}" | head -n 20
else:
    # 2. 파일은 있는데 데이터가 안 뽑히는 경우 -> 구조 확인
    print("\n✅ 파일은 발견했습니다. 첫 번째 파일의 구조를 뜯어봅니다.")
    target_file = all_files[0]
    print(f"분석 대상 파일: {target_file}")

    with open(target_file, 'r', encoding='utf-8') as f:
        try:
            content = json.load(f)
            print(f"최상위 데이터 타입: {type(content)}")

            # 딕셔너리인 경우 키값 확인
            if isinstance(content, dict):
                print(f"최상위 키(Keys): {list(content.keys())}")
                # 만약 'data' 같은 키 안에 리스트가 있다면 그 안도 확인
                for key in content.keys():
                    if isinstance(content[key], list) and len(content[key]) > 0:
                        print(f"Key '{key}' 내부 첫 번째 아이템 샘플: {content[key][0].keys()}")
                        break
            # 리스트인 경우 첫 번째 아이템 확인
            elif isinstance(content, list) and len(content) > 0:
                print(f"리스트 내부 첫 번째 아이템 키(Keys): {content[0].keys()}")
            else:
                print("데이터가 비어있거나 예상치 못한 구조입니다.")

        except Exception as e:
            print(f"JSON 읽기 에러: {e}")

현재 설정된 DATA_DIR: data/쇼핑몰/02. 화장품
발견된 JSON 파일 개수: 450

✅ 파일은 발견했습니다. 첫 번째 파일의 구조를 뜯어봅니다.
분석 대상 파일: data/쇼핑몰/02. 화장품/2-3.메이크업뷰티소품/2-3.메이크업뷰티소품(43).json
최상위 데이터 타입: <class 'list'>
리스트 내부 첫 번째 아이템 키(Keys): dict_keys(['Index', 'RawText', 'Source', 'Domain', 'MainCategory', 'ProductName', 'ReviewScore', 'Syllable', 'Word', 'RDate', 'GeneralPolarity', 'Aspects'])


## 3. 전처리 (Preprocessing)

In [ ]:
# ---------------------------------------------------------
# 5. 토크나이징 및 전처리
# ---------------------------------------------------------
MODEL_ID = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, padding=False)

# 여기서 dataset 변수가 정상적으로 사용됩니다
encoded_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Preprocessing Complete.")

# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Map:   0%|          | 0/1638 [00:00<?, ? examples/s]

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

Preprocessing Complete.


## 4. Full Fine-Tuning

In [ ]:
model_ft = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=3)

training_args = TrainingArguments(
    output_dir="./results/full_ft",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3 if not is_mac_local else 1,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(), # Use Mixed Precision on CUDA
    report_to="none",
    gradient_accumulation_steps=2,
)

trainer_ft = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

if is_mac_local:
    print("!!! Local Debugging Mode: Skipping Full Training !!!")
    # trainer_ft.train() # Uncomment to debug run
else:
    trainer_ft.train()
    trainer_ft.save_model("./saved_models/full_ft")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-547381384.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ft = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.671717,0.742291,0.614400
2,No log,0.386439,0.856828,0.822034
3,No log,0.395527,0.852423,0.805895


## 5. PEFT (LoRA)

In [ ]:
# Reset to base model for Fairness
model_lora = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=3)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8, # Rank
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"] # Target Attention Layers
)

model_lora = get_peft_model(model_lora, lora_config)
model_lora.print_trainable_parameters()

training_args_lora = TrainingArguments(
    output_dir="./results/lora",
    learning_rate=1e-4, # Higher LR for LoRA
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3 if not is_mac_local else 1,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

if is_mac_local:
    print("!!! Local Debugging Mode: Skipping LoRA Training !!!")
else:
    trainer_lora.train()
    trainer_lora.save_model("./saved_models/lora_ft")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-641167954.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


trainable params: 887,811 || all params: 111,508,230 || trainable%: 0.7962


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.751021,0.588106,0.246879
2,No log,0.457338,0.797357,0.722738
3,No log,0.438757,0.821586,0.756631


## 6. 비교 분석 및 결과 Summary

> 인용구 추가


- Full FT와 LoRA의 F1 Score, Training Time, Model Size를 비교합니다.

In [ ]:
import os
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import torch

# 결과 저장 폴더 생성
RESULTS_DIR = "./results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# ---------------------------------------------------------
# 1. 로그 데이터 수집 (사용자님의 로그 기반 하드코딩)
# ---------------------------------------------------------
# Full FT: 1m 20s = 80s, Best F1: 0.8220 (Epoch 2)
# LoRA: 33s, Best F1: 0.7566 (Epoch 3)

comparison_data = {
    "Model": ["Full Fine-Tuning", "LoRA"],
    "Best F1 Score": [0.8220, 0.7566],
    "Best Accuracy": [0.8568, 0.8215],
    "Training Time (sec)": [80, 33],        # 1분 20초 vs 33초
    "Trainable Params": [111508230, 887811] # 약 1.1억개 vs 88만개
}

df_results = pd.DataFrame(comparison_data)

# 학습 곡선 데이터 (Epoch별 F1 변화)
history_data = {
    "Epoch": [1, 2, 3],
    "Full FT F1": [0.6144, 0.8220, 0.8058],
    "LoRA F1":    [0.2468, 0.7227, 0.7566]
}
df_history = pd.DataFrame(history_data)

# ---------------------------------------------------------
# 2. 정량적 분석: Plotly 시각화
# ---------------------------------------------------------
print("📊 결과 시각화를 생성합니다...")

# (1) 성능(F1) vs 속도(Time) 비교 차트
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

# Bar: F1 Score
fig1.add_trace(
    go.Bar(x=df_results["Model"], y=df_results["Best F1 Score"], name="Best F1 Score", marker_color=['#636EFA', '#EF553B']),
    secondary_y=False
)

# Line: Training Time
fig1.add_trace(
    go.Scatter(x=df_results["Model"], y=df_results["Training Time (sec)"], name="Training Time (s)", mode='lines+markers', line=dict(color='green', dash='dot')),
    secondary_y=True
)

fig1.update_layout(title_text="<b>[Performance vs Efficiency]</b> Full FT vs LoRA")
fig1.update_yaxes(title_text="F1 Score", secondary_y=False, range=[0, 1.0])
fig1.update_yaxes(title_text="Training Time (sec)", secondary_y=True)

# (2) 학습 곡선 비교 (Epochs vs F1)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_history["Epoch"], y=df_history["Full FT F1"], mode='lines+markers', name='Full FT'))
fig2.add_trace(go.Scatter(x=df_history["Epoch"], y=df_history["LoRA F1"], mode='lines+markers', name='LoRA'))
fig2.update_layout(title="<b>[Learning Curve]</b> Epoch vs F1 Score", xaxis_title="Epoch", yaxis_title="F1 Score")

# (3) 파라미터 효율성 (Log Scale Bar)
fig3 = px.bar(df_results, x="Model", y="Trainable Params", title="<b>[Parameter Efficiency]</b> Trainable Parameters (Log Scale)", log_y=True, text="Trainable Params", color="Model")
fig3.update_traces(texttemplate='%{text:.2s}', textposition='outside')

# 차트 출력
fig1.show()
fig2.show()
fig3.show()

# 차트 저장 (HTML)
fig1.write_html(f"{RESULTS_DIR}/comparison_performance.html")
fig2.write_html(f"{RESULTS_DIR}/comparison_learning_curve.html")
print(f"✅ 차트 저장 완료: {RESULTS_DIR}/")

# ---------------------------------------------------------
# 3. 정성적 분석: 실제 추론 결과 비교 (Inference)
# ---------------------------------------------------------
print("\n🔍 [정성적 분석] 실제 문장 추론 테스트")

test_sentence = "배송도 정말 빠르고 디자인도 사진이랑 똑같아요! 다만 마감이 살짝 아쉽네요."
# 예상: 긍정(2) 혹은 중립(1). '마감이 아쉽다'는 부정 요소가 섞여 있어 모델의 판단이 중요함.

def predict_sentiment(model, text):
    model.eval()
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    top_prob, top_label = torch.max(probs, dim=-1)
    return top_label.item(), top_prob.item()

# Label Map
id2label = {0: "부정(Negative)", 1: "중립(Neutral)", 2: "긍정(Positive)"}

# Full FT 예측
ft_label, ft_conf = predict_sentiment(trainer_ft.model, test_sentence)
# LoRA 예측
lora_label, lora_conf = predict_sentiment(trainer_lora.model, test_sentence)

print(f"\n📝 테스트 문장: '{test_sentence}'")
print("-" * 50)
print(f"🤖 Full FT 모델 예측 : {id2label[ft_label]} (확신도: {ft_conf*100:.2f}%)")
print(f"⚡ LoRA 모델 예측    : {id2label[lora_label]} (확신도: {lora_conf*100:.2f}%)")
print("-" * 50)

📊 결과 시각화를 생성합니다...


✅ 차트 저장 완료: ./results/

🔍 [정성적 분석] 실제 문장 추론 테스트

📝 테스트 문장: '배송도 정말 빠르고 디자인도 사진이랑 똑같아요! 다만 마감이 살짝 아쉽네요.'
--------------------------------------------------
🤖 Full FT 모델 예측 : 중립(Neutral) (확신도: 67.14%)
⚡ LoRA 모델 예측    : 긍정(Positive) (확신도: 74.81%)
--------------------------------------------------


In [ ]:
import pandas as pd

def extract_best_metrics(trainer, name):
    # 로그 히스토리에서 'eval_f1'이 있는 기록만 가져오기
    eval_logs = [log for log in trainer.state.log_history if 'eval_f1' in log]

    # F1 점수가 가장 높은 기록 찾기
    if eval_logs:
        best_log = max(eval_logs, key=lambda x: x['eval_f1'])
        best_f1 = best_log['eval_f1']
        best_acc = best_log['eval_accuracy']
        best_epoch = best_log['epoch']
    else:
        best_f1, best_acc, best_epoch = 0, 0, 0

    # 총 학습 시간 (로그의 마지막 부분에 보통 기록됨)
    # 없을 경우를 대비해 안전하게 처리
    try:
        train_runtime = trainer.state.log_history[-1].get('train_runtime', 0)
    except:
        train_runtime = 0

    # 파라미터 수 계산
    total_params = sum(p.numel() for p in trainer.model.parameters())
    trainable_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)

    return {
        "Model": name,
        "Best F1": best_f1,
        "Best Accuracy": best_acc,
        "Best Epoch": best_epoch,
        "Training Time(s)": round(train_runtime, 2),
        "Trainable Params": f"{trainable_params:,} ({(trainable_params/total_params)*100:.1f}%)"
    }

# 1. 실제 데이터 추출
res_ft = extract_best_metrics(trainer_ft, "Full Fine-Tuning")
res_lora = extract_best_metrics(trainer_lora, "LoRA")

# 2. DataFrame으로 보기 좋게 출력
df_real_result = pd.DataFrame([res_ft, res_lora])
display(df_real_result)

,Model,Best F1,Best Accuracy,Best Epoch,Training Time(s),Trainable Params
0,Full Fine-Tuning,0.822034,0.856828,2.0,84.38,"110,620,419 (100.0%)"
1,LoRA,0.756631,0.821586,3.0,34.15,"887,811 (0.8%)"


Learning Curve (Fig 2) 확인:

Full FT는 1 Epoch부터 이미 성능이 높게 나옵니다.

LoRA는 1 Epoch에서 성능이 매우 낮았다가(0.24) 2 Epoch부터 급격히 따라잡는 모습을 보입니다. 이는 LoRA가 초기 가중치 정렬에 시간이 조금 더 필요함을 의미합니다. Epoch를 5~10회로 늘리면 Full FT와 거의 비슷해질 가능성이 높습니다.

정성적 분석 결과:

test_sentence("배송 빠르고... 마감이 아쉽다")는 복합적인 감정이 섞여 있습니다. 두 모델이 미세하게 다른 확신도(Confidence)를 보이는지 확인해보세요.

In [ ]:
import torch
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch.nn.functional as F

# 1. 다양한 테스트 문장 리스트 (난이도별 배치)
test_sentences = [
    "배송도 빠르고 디자인도 너무 마음에 들어요! 강추합니다.",            # [명확한 긍정]
    "최악이에요. 다신 안 삽니다. 환불해주세요.",                       # [명확한 부정]
    "그냥 평범해요. 가격 대비 쏘쏘한 느낌?",                           # [중립/애매]
    "디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요.",             # [복합: 긍정+부정 -> 보통 부정이나 중립]
    "배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요.",    # [복합: 부정->긍정]
    "도대체 언제 도착하나요? 일주일째 기다리고 있습니다."                  # [문의/불만 -> 부정]
]

labels_map = {0: "부정(Neg)", 1: "중립(Neu)", 2: "긍정(Pos)"}

# 2. 확률 추출 함수 (Softmax 적용)
def get_probabilities(model, text, tokenizer, device):
    model.eval()
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Logits -> Probabilities (0~1 사이 값으로 변환)
    probs = F.softmax(outputs.logits, dim=-1)[0].cpu().numpy()
    return probs # [prob_neg, prob_neu, prob_pos]

# 3. Full FT vs LoRA 추론 실행
results = []

print("🔎 상세 추론 분석을 진행합니다...")

for text in test_sentences:
    # Full FT 예측
    ft_probs = get_probabilities(trainer_ft.model, text, tokenizer, device)
    # LoRA 예측
    lora_probs = get_probabilities(trainer_lora.model, text, tokenizer, device)

    results.append({
        "문장": text,
        # Full FT 결과
        "FT_Neg": ft_probs[0], "FT_Neu": ft_probs[1], "FT_Pos": ft_probs[2],
        "FT_Pred": labels_map[ft_probs.argmax()],
        # LoRA 결과
        "LoRA_Neg": lora_probs[0], "LoRA_Neu": lora_probs[1], "LoRA_Pos": lora_probs[2],
        "LoRA_Pred": labels_map[lora_probs.argmax()]
    })

df_analysis = pd.DataFrame(results)

# 4. 결과 테이블 출력 (가독성 좋게 퍼센트로 변환)
pd.options.display.float_format = '{:.1%}'.format
print("\n📊 [모델별 클래스 확률 분포 비교]")
display(df_analysis[["문장", "FT_Pred", "FT_Neg", "FT_Neu", "FT_Pos", "LoRA_Pred", "LoRA_Neg", "LoRA_Neu", "LoRA_Pos"]])

# 5. 시각화 (Plotly Grouped Bar Chart)
print("\n📈 시각화 생성 중...")

# 복잡한 그래프를 위해 데이터 구조 변경
fig = make_subplots(
    rows=len(test_sentences), cols=1,
    subplot_titles=[f"Q{i+1}: {t[:20]}..." for i, t in enumerate(test_sentences)],
    vertical_spacing=0.05
)

colors = {'부정': '#EF553B', '중립': '#FECB52', '긍정': '#00CC96'} # Red, Yellow, Green

for i, row in df_analysis.iterrows():
    # Full FT Bars
    fig.add_trace(go.Bar(
        name='Full FT',
        x=['부정', '중립', '긍정'],
        y=[row['FT_Neg'], row['FT_Neu'], row['FT_Pos']],
        marker_color=[colors['부정'], colors['중립'], colors['긍정']],
        opacity=0.9,
        text=[f"{v:.1%}" for v in [row['FT_Neg'], row['FT_Neu'], row['FT_Pos']]],
        textposition='auto',
        legendgroup='Full FT',
        showlegend=(i==0) # 범례는 처음에만 표시
    ), row=i+1, col=1)

    # LoRA Bars (비교를 위해 옆에 얇게 혹은 겹쳐서 표현하고 싶으나, 여기선 별도 Trace로)
    # 비교를 명확히 하기 위해 LoRA는 투명도 조절 or 패턴 사용
    fig.add_trace(go.Bar(
        name='LoRA',
        x=['부정', '중립', '긍정'],
        y=[row['LoRA_Neg'], row['LoRA_Neu'], row['LoRA_Pos']],
        marker_color=[colors['부정'], colors['중립'], colors['긍정']],
        opacity=0.4, # LoRA는 흐리게 표시해서 겹쳐 보이게 함
        text=[f"{v:.1%}" for v in [row['LoRA_Neg'], row['LoRA_Neu'], row['LoRA_Pos']]],
        textposition='outside',
        legendgroup='LoRA',
        showlegend=(i==0)
    ), row=i+1, col=1)

fig.update_layout(
    title_text="<b>[심층 분석] 문장별 감정 확률 분포 (진한색: Full FT / 연한색: LoRA)</b>",
    height=300 * len(test_sentences),
    barmode='group' # 그룹으로 묶어서 비교
)

fig.show()

🔎 상세 추론 분석을 진행합니다...

📊 [모델별 클래스 확률 분포 비교]


,문장,FT_Pred,FT_Neg,FT_Neu,FT_Pos,LoRA_Pred,LoRA_Neg,LoRA_Neu,LoRA_Pos
0,배송도 빠르고 디자인도 너무 마음에 들어요! 강추합니다.,긍정(Pos),0.7%,1.7%,97.6%,긍정(Pos),0.1%,0.9%,99.0%
1,최악이에요. 다신 안 삽니다. 환불해주세요.,부정(Neg),86.8%,5.8%,7.4%,부정(Neg),85.0%,6.5%,8.5%
2,그냥 평범해요. 가격 대비 쏘쏘한 느낌?,중립(Neu),27.2%,63.5%,9.2%,중립(Neu),26.7%,58.4%,14.9%
3,"디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요.",부정(Neg),64.1%,31.6%,4.3%,부정(Neg),70.6%,22.4%,7.0%
4,"배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요.",긍정(Pos),1.0%,11.0%,88.0%,긍정(Pos),6.6%,27.6%,65.8%
5,도대체 언제 도착하나요? 일주일째 기다리고 있습니다.,부정(Neg),69.9%,24.0%,6.0%,부정(Neg),89.8%,7.3%,2.9%



📈 시각화 생성 중...


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix

# 1. 전체 테스트 데이터셋에 대해 추론(Prediction) 실행
print("🚀 전체 테스트 데이터셋에 대해 추론을 시작합니다...")

# Full FT 예측
pred_output_ft = trainer_ft.predict(encoded_dataset["test"])
pred_labels_ft = np.argmax(pred_output_ft.predictions, axis=1)

# LoRA 예측
pred_output_lora = trainer_lora.predict(encoded_dataset["test"])
pred_labels_lora = np.argmax(pred_output_lora.predictions, axis=1)

# 실제 정답 (Ground Truth)
true_labels = np.array(encoded_dataset["test"]["label"])

# 라벨 매핑
label_map = {0: '부정', 1: '중립', 2: '긍정'}
labels_list = ['부정', '중립', '긍정']

# ---------------------------------------------------------
# 2. 분포 비교 (Distribution Comparison)
# ---------------------------------------------------------
# 각 리스트를 데이터프레임으로 변환하여 카운팅
df_dist = pd.DataFrame({
    'Actual': [label_map[l] for l in true_labels],
    'Full FT': [label_map[l] for l in pred_labels_ft],
    'LoRA': [label_map[l] for l in pred_labels_lora]
})

# 비율 계산 함수
def get_distribution(series, name):
    counts = series.value_counts(normalize=True).reindex(labels_list, fill_value=0) * 100
    df = counts.reset_index()
    df.columns = ['Sentiment', 'Percentage']
    df['Model'] = name
    return df

dist_actual = get_distribution(df_dist['Actual'], '실제 정답(Ground Truth)')
dist_ft = get_distribution(df_dist['Full FT'], 'Full FT 예측')
dist_lora = get_distribution(df_dist['LoRA'], 'LoRA 예측')

# 하나로 합치기
df_final_dist = pd.concat([dist_actual, dist_ft, dist_lora])

# 시각화 1: 분포 비교 막대 그래프
fig_dist = px.bar(
    df_final_dist,
    x="Sentiment",
    y="Percentage",
    color="Model",
    barmode="group",
    text_auto='.1f',
    title="<b>[전체 분포 비교] 실제 정답 vs 모델별 예측 비율</b>",
    color_discrete_map={'실제 정답(Ground Truth)': 'gray', 'Full FT 예측': '#636EFA', 'LoRA 예측': '#EF553B'}
)
fig_dist.update_layout(yaxis_title="비율 (%)")
fig_dist.show()

# ---------------------------------------------------------
# 3. 오답 분석 (Confusion Matrix)
# ---------------------------------------------------------
# 혼동 행렬 계산
cm_ft = confusion_matrix(true_labels, pred_labels_ft)
cm_lora = confusion_matrix(true_labels, pred_labels_lora)

# 시각화 2: Confusion Matrix Heatmap
fig_cm = make_subplots(
    rows=1, cols=2,
    subplot_titles=("<b>Full FT 혼동 행렬</b>", "<b>LoRA 혼동 행렬</b>"),
    horizontal_spacing=0.15
)

# Full FT Heatmap
fig_cm.add_trace(
    go.Heatmap(
        z=cm_ft, x=labels_list, y=labels_list,
        colorscale='Blues', text=cm_ft, texttemplate="%{text}", textfont={"size":16},
        name="Full FT"
    ), row=1, col=1
)

# LoRA Heatmap
fig_cm.add_trace(
    go.Heatmap(
        z=cm_lora, x=labels_list, y=labels_list,
        colorscale='Reds', text=cm_lora, texttemplate="%{text}", textfont={"size":16},
        name="LoRA"
    ), row=1, col=2
)

fig_cm.update_layout(
    title_text="<b>[오답 상세 분석] Confusion Matrix (X: 예측, Y: 실제)</b>",
    xaxis_title="모델의 예측 (Predicted)", yaxis_title="실제 정답 (Actual)",
    xaxis2_title="모델의 예측 (Predicted)", yaxis2_title="실제 정답 (Actual)"
)
fig_cm.show()

# ---------------------------------------------------------
# 4. 요약 텍스트 출력
# ---------------------------------------------------------
print("\n📝 [분석 코멘트 생성]")
print("-" * 50)
print(f"총 테스트 데이터 수: {len(true_labels)}개")

# 정확도 재확인
acc_ft = np.mean(pred_labels_ft == true_labels)
acc_lora = np.mean(pred_labels_lora == true_labels)

print(f"Full FT 정확도: {acc_ft:.2%}")
print(f"LoRA    정확도: {acc_lora:.2%}")

# 편향성 간단 체크
cnt_actual = df_dist['Actual'].value_counts()
cnt_ft = df_dist['Full FT'].value_counts()
cnt_lora = df_dist['LoRA'].value_counts()

print("\n[감정별 개수 비교 (실제 vs FT vs LoRA)]")
for label in labels_list:
    a = cnt_actual.get(label, 0)
    f = cnt_ft.get(label, 0)
    l = cnt_lora.get(label, 0)
    print(f"- {label}: 실제 {a}개 | FT {f}개 ({f-a:+d}) | LoRA {l}개 ({l-a:+d})")
print("-" * 50)

🚀 전체 테스트 데이터셋에 대해 추론을 시작합니다...



📝 [분석 코멘트 생성]
--------------------------------------------------
총 테스트 데이터 수: 454개
Full FT 정확도: 85.68%
LoRA    정확도: 82.16%

[감정별 개수 비교 (실제 vs FT vs LoRA)]
- 부정: 실제 88개 | FT 81개 (-7) | LoRA 103개 (+15)
- 중립: 실제 99개 | FT 106개 (+7) | LoRA 70개 (-29)
- 긍정: 실제 267개 | FT 267개 (+0) | LoRA 281개 (+14)
--------------------------------------------------


# 📊 [최종 리포트] 감정 분석 모델 성능 비교: Full Fine-Tuning vs. LoRA

## 1. 프로젝트 개요
- **목적**: 한국어 쇼핑몰 리뷰 감정 분석 (3-Class: 부정, 중립, 긍정)
- **비교 대상**:
  1. **Full Fine-Tuning (FT)**: 모델 전체 파라미터 업데이트 (전통적 방식)
  2. **LoRA (Low-Rank Adaptation)**: 핵심 가중치만 학습 (경량화 방식)
- **데이터셋**: Train 1,638건 / Test 454건

---

## 2. 정량적 성과 요약 (Quantitative Summary)

| 비교 항목 | Full Fine-Tuning | LoRA | 비고 |
|:---:|:---:|:---:|:---|
| **Best Accuracy** | **85.68%** | 82.16% | FT가 약 **3.5%p** 우세 |
| **Best F1 Score** | **0.8220** | 0.7566 | FT가 균형 잡힌 성능 보임 |
| **학습 시간** | 84.38초 | **34.15초** | LoRA가 약 **2.5배 빠름** |
| **학습 파라미터** | 1.1억 개 (100%) | **88만 개 (0.8%)** | LoRA가 메모리 효율 압도적 |
| **Best Epoch** | 2 Epoch | 3 Epoch | LoRA는 학습 수렴이 다소 느림 |

> **요약**: 정확도는 Full FT가 더 높지만, LoRA는 단 **0.8%의 파라미터**만으로 FT 성능의 **96% 수준**까지 도달하는 효율성을 증명함.

---

## 3. 예측 분포 및 편향성 분석 (Distribution Analysis)
테스트 데이터 454개에 대한 두 모델의 예측 성향을 분석했습니다.

| 감정 클래스 | 실제 정답(Ground Truth) | Full FT 예측 | LoRA 예측 | 분석 (Insight) |
|:---:|:---:|:---:|:---:|:---|
| **부정 (Negative)** | 88개 | 81개 (-7) | **103개 (+15)** | LoRA는 부정을 다소 과하게 예측함 |
| **중립 (Neutral)** | 99개 | **106개 (+7)** | **70개 (-29)** | **LoRA의 약점**. 애매한 리뷰를 중립으로 보지 못함 |
| **긍정 (Positive)** | 267개 | 267개 (0) | 281개 (+14) | 두 모델 모두 긍정은 잘 맞춤 |

- **Full FT**: 실제 분포와 매우 유사하게 예측하며, 특히 **중립(Neutral)** 클래스를 잘 잡아냄.
- **LoRA**: **'양극화 현상'**이 관찰됨. 애매한(중립) 데이터를 **부정이나 긍정으로 쏠려서 판단**하는 경향이 있음. (중립 → 부정/긍정 오분류 증가)

---

## 4. 정성적 분석 (Case Study)
복합적인 감정이 섞인 문장에 대해 모델이 어떻게 반응하는지 확인했습니다.

### ① 복합 감정 처리 ("배송 늦음(부정) + 옷 예쁨(긍정)")
> *문장: "배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요."*
- **Full FT**: **긍정 88%** (확신 높음)
- **LoRA**: **긍정 65%** / 중립 27% (다소 주저함)
- **해석**: 문맥상 '용서가 된다'는 긍정 결론을 FT가 더 명확하게 파악했습니다. LoRA는 배송 지연 키워드 때문에 중립 확률이 섞였습니다.

### ② 성능 불만 처리 ("디자인 예쁨(긍정) + 마감 엉망(부정)")
> *문장: "디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요."*
- **Full FT**: **부정 64%**
- **LoRA**: **부정 70%**
- **해석**: 두 모델 모두 '실망'이라는 부정 키워드에 높은 가중치를 두어 정답(부정)을 잘 맞췄습니다. 의외로 LoRA가 부정 판단은 더 과감했습니다.

### ③ 명확한 문장 처리
- 단순 긍정/부정 문장에서는 두 모델 모두 **85%~99%**의 높은 확신도로 정답을 맞췄습니다.

---

## 5. 결론 및 제언 (Conclusion)

### ✅ 모델 선정 가이드
1. **최고의 성능(Accuracy)이 최우선이라면?** 👉 **Full Fine-Tuning**
   - 특히 **'중립'**이나 **'복합적인 뉘앙스'**를 정확히 구분해야 하는 서비스에 적합합니다.
   
2. **GPU 자원이 부족하거나 빠른 배포가 필요하다면?** 👉 **LoRA**
   - 학습 속도가 2.5배 빠르고 모델 용량이 매우 작습니다.
   - 단, **Epoch를 더 늘리거나(5~10회)**, Rank(r) 값을 조정하여 '중립' 클래스 학습을 보강할 필요가 있습니다.

### 🚀 향후 발전 방향
- **LoRA 튜닝**: 현재 LoRA가 중립을 잘 못 맞추므로, `lora_alpha`나 `r` 값을 높여 표현력을 증대시킬 필요가 있음.
- **데이터 증강**: 중립(Neutral) 데이터가 상대적으로 적으므로, 관련 데이터를 추가 확보하면 성능 향상 기대.